In [2]:
!pip install streamlit -q
!pip install pyngrok -q

In [5]:
%%writefile app.py
import streamlit as st
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
import torch

# モデルとトークナイザーの読み込み
MODEL_PATH = "/kaggle/input/deepseek-math"
config = AutoConfig.from_pretrained(MODEL_PATH)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

# モデルの読み込み
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype="auto",
    trust_remote_code=True,
    config=config
)

# StreamlitアプリのUI
st.title('Math Problem Solver')
prompt = st.text_input('Enter a math problem:', 'Solve the following math problem: What is the integral of x^2?')

if st.button('Solve'):
    # プロンプトをトークン化
    inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
    
    # テキスト生成
    output = model.generate(
        **inputs,
        max_new_tokens=50,  # 生成するトークン数の最大値
        do_sample=True,
        temperature=0.9,
        top_p=0.95
    )
    
    # 生成されたテキストをデコード
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    st.write(generated_text)


Overwriting app.py


In [3]:
%%writefile app.py
import streamlit as st
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
import torch
import numpy as np
from numpy.random import choice

# モデルとトークナイザーの読み込み
MODEL_PATH = "/kaggle/input/deepseek-math"
config = AutoConfig.from_pretrained(MODEL_PATH)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

# モデルの読み込み
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype="auto",
    trust_remote_code=True,
    config=config
)

# プロンプトテンプレート
code_template = """Below is a math problem you are to solve (positive numerical answer):
"{}"
To accomplish this, first determine a sympy-based approach for solving the problem by listing each step to take and what functions need to be called in each step. Be clear so even an idiot can follow your instructions, and remember, your final answer should be positive integer, not an algebraic expression!
Write the entire script covering all the steps (use comments and document it well) and print the result. After solving the problem, output the final numerical answer within \\boxed{}.

Approach:"""

cot_template = """Below is a math problem you are to solve (positive numerical answer!):
"{}"
Analyze this problem and think step by step to come to a solution with programs. After solving the problem, output the final numerical answer within \\boxed{}.\n\n"""

prompt_options = [code_template, cot_template]

# StreamlitアプリのUI
st.title('Math Problem Solver')
problem = st.text_input('Enter a math problem:', 'Find the units digit of \( 7^{100} \).')

if st.button('Solve'):
    TOTAL_TOKENS = 2048
    ALREADY_GEN = 0
    question_type_counts = {}
    starting_counts = (2, 3)
    i = 0
    temperature = 0.9
    top_p = 1.0

    text_answers, code_answers = question_type_counts.get(i, starting_counts)
    counts = np.array([text_answers, code_answers])
    draw = choice(prompt_options, 1, p=counts/counts.sum())

    initial_message = draw[0].format(problem, "{}")
    prompt = f"User: {initial_message}"
    print(prompt)
    
    model_inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
    generation_output = model.generate(**model_inputs, 
                                       max_new_tokens=TOTAL_TOKENS-ALREADY_GEN,
                                       do_sample=True,
                                       temperature=temperature,
                                       top_p=top_p,
                                       num_return_sequences=1)
    
    generated_text = tokenizer.decode(generation_output[0], skip_special_tokens=True)
    st.write(generated_text)

Writing app.py


In [5]:
import subprocess
from pyngrok import ngrok

# ngrokの認証トークンを設定します
NGROK_AUTH_TOKEN = "2dTzmleYxq1mTfiDFbkxE1wvHG3_2cw3k3t36UDyDFUr29TUz"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Streamlitアプリをサブプロセスとして実行します
streamlit_process = subprocess.Popen(['streamlit', 'run', 'app.py'])

# ngrokトンネルを作成して、ポート8501を指定します
public_url = ngrok.connect(8501)
print(f'Streamlit app is live at: {public_url}')

Streamlit app is live at: NgrokTunnel: "https://1d5a-35-184-78-8.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.19.2.2:8501
  External URL: http://35.184.78.8:8501



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 3/3 [01:14<00:00, 24.80s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 3/3 [01:09<00:00, 23.15s/it]
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


In [6]:
streamlit_process.terminate()

User: Below is a math problem you are to solve (positive numerical answer!):
"Find the units digit of \( 7^{100} \)."
Analyze this problem and think step by step to come to a solution with programs. After solving the problem, output the final numerical answer within \boxed{}.


  Stopping...


aaaa